In [11]:
import csv
import os
import numpy as np
from sklearn import tree
from sklearn.tree import export_text

root_data_dir = "raw_data_2023_0718"
header = []

def get_avg(path):
    with open(path, 'r', encoding='utf-8') as file:
        reader = csv.reader(file)
        # 跳过表头
        global header 
        header = next(reader)[1:]
        
        # 跳过时间戳
        data = []
        for row in reader:
            # 跳过含有空值的行
            if len(row) > 0 and all(cell.strip() != '' for cell in row):
                # 去除时间戳
                data.append([float(e) for e in row[1:]])
                
    column_means = np.mean(np.array(data), axis=0)
    return column_means

def normalize(matrix):
    matrix_std = np.std(matrix, axis=0)
    matrix_mean = np.mean(matrix, axis=0)

    zero_std_cols = np.where(matrix_std == 0)[0]

    matrix_mean[zero_std_cols] = 0
    matrix_std[zero_std_cols] = 1

    return (matrix - matrix_mean) / matrix_std

In [12]:
app_data_dirs = [os.path.join(root_data_dir, dir_path) for dir_path in os.listdir(root_data_dir)]

X = []
Y = []
app_names = []
for data_dir in app_data_dirs:
    if not os.path.isdir(data_dir):
        continue
    for file in os.listdir(data_dir):
        file_path = os.path.join(data_dir, file)
        if not os.path.isfile(file_path):
            continue
        X.append(get_avg(file_path))
        Y.append(len(app_names))
        
    app_names.append(os.path.basename(data_dir))

# no need for dts
# X = normalize(X)

In [13]:
clf = tree.DecisionTreeClassifier()
clf = clf.fit(X, Y)

r = export_text(clf, feature_names=header, class_names=app_names)
print(r)

|--- host_network_io_bytes_transmit <= 2721717.50
|   |--- class: mysql
|--- host_network_io_bytes_transmit >  2721717.50
|   |--- class: redis

